# 📊 图像色彩分析报告模块（可视化 + 经纬度 + 聚类 + Excel 导出）

## 🗺️ 经纬度提取（从文件名中提取经纬度）

In [ ]:
import pandas as pd
import re

df = pd.read_csv("F:/SCI essays/Shenzhen Street View/Color Results/color_analysis_all.csv")

def extract_coordinates(filename):
    match = re.search(r"(\d+)[_,](\d+\.\d+),(\d+\.\d+)", filename)
    if match:
        return float(match.group(2)), float(match.group(3))
    return None, None

df['longitude'], df['latitude'] = zip(*df['image'].map(extract_coordinates))
df.dropna(inplace=True)
df.head()


## 📊 色彩指标分布图

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
for i, col in enumerate(["color_complexity", "color_entropy", "color_harmony"]):
    plt.subplot(1, 3, i+1)
    df[col].hist(bins=30)
    plt.title(col)
plt.tight_layout()
plt.show()


## 🌍 色彩熵地理分布图

In [ ]:
plt.figure(figsize=(10, 6))
sc = plt.scatter(df['longitude'], df['latitude'], c=df['color_entropy'], cmap='viridis', s=10)
plt.colorbar(sc, label="Color Entropy")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Color Entropy Geographic Distribution")
plt.show()


## 📈 KMeans 聚类分析

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

features = df[['color_complexity', 'color_entropy', 'color_harmony']]
scaled = StandardScaler().fit_transform(features)
kmeans = KMeans(n_clusters=4, random_state=0).fit(scaled)
df['cluster'] = kmeans.labels_

plt.figure(figsize=(8, 6))
plt.scatter(df['color_entropy'], df['color_complexity'], c=df['cluster'], cmap='tab10')
plt.xlabel("Color Entropy")
plt.ylabel("Color Complexity")
plt.title("Color Feature Clustering")
plt.show()


## 📤 报告导出为 Excel

In [ ]:
df.to_excel("F:/SCI essays/Shenzhen Street View/Color Results/color_analysis_report.xlsx", index=False)
print("导出成功：color_analysis_report.xlsx")
